In [1]:
import pandas as pd
from datetime import datetime

In [2]:
TWEETS_812A_JAN = "data/tweets/812A_2019_JAN_2KM.csv"
TWEETS_812A_FEV = "data/tweets/812A_2019_FEV_2KM.csv"
TWEETS_812A_MAR = "data/tweets/812A_2019_MAR_2KM.csv"
TWEETS_833A = "data/tweets/833A_2019_2KM.csv"

In [3]:
RAIN_GAUGE_CODES = ["355030812A", "355030833A"]
RAIN_GAUGES_JAN = "data/rain_gauges/data1.csv"
RAIN_GAUGES_FEV = "data/rain_gauges/data2.csv"
RAIN_GAUGES_MAR = "data/rain_gauges/data3.csv"

In [4]:
FLOODS_812A = "data/floods/812A_alags.csv"
FLOODS_833A = "data/floods/833A_alags.csv"
GEO_812A = [-46.632533, -23.531015]
GEO_833A = [-46.635987, -23.543311]

In [5]:
KEYWORDS = ["chuva", "chove", "chuvoso", "chuvosa", "chuvarada", 
            "alagamento", "alagou", "inundação", "inundada", "enxurrada",
            "rio transbordou", "córrego transbordou", "rio saiu da calha", 
            "córrego saiu da calha", "deslizamento", "escorregamento", "queda de barreira"]

## Time range creation

In [6]:
DAYS_RANGE = datetime(2019, 4, 1) - datetime(2019, 1, 1) 
TIME_PERIOD = DAYS_RANGE.days * 24 * 6

In [7]:
begin_date = datetime.strptime("01-01-2019", "%d-%m-%Y")

In [8]:
date_range = pd.date_range(begin_date, periods=TIME_PERIOD, freq="10min")

In [9]:
date_range = date_range.to_frame(index=False, name="time")

## Rain Gauge Data

In [10]:
rain_gauges_jan = pd.read_csv(RAIN_GAUGES_JAN, sep=";", index_col=False, decimal=".")
rain_gauges_fev = pd.read_csv(RAIN_GAUGES_FEV, sep=";", index_col=False)
rain_gauges_mar = pd.read_csv(RAIN_GAUGES_MAR, sep=";", index_col=False)

In [11]:
rain_gauges = pd.concat([rain_gauges_jan, rain_gauges_fev, rain_gauges_mar])

In [12]:
filtered_rain_gauges = rain_gauges[rain_gauges["codEstacao"].isin(RAIN_GAUGE_CODES)]

In [13]:
filtered_rain_gauges["datahora"] = pd.to_datetime(filtered_rain_gauges["datahora"])
filtered_rain_gauges["valorMedida"] = filtered_rain_gauges["valorMedida"].str.replace(",", ".")
filtered_rain_gauges["valorMedida"] = pd.to_numeric(filtered_rain_gauges["valorMedida"])

C:\Users\Jefferson\AppData\Local\Temp\ipykernel_18652\3252865042.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rain_gauges["datahora"] = pd.to_datetime(filtered_rain_gauges["datahora"])
C:\Users\Jefferson\AppData\Local\Temp\ipykernel_18652\3252865042.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rain_gauges["valorMedida"] = filtered_rain_gauges["valorMedida"].str.replace(",", ".")
C:\Users\Jefferson\AppData\Local\Temp\ipykernel_18652\3252865042.py:3: SettingWithCopyWarning: 
A 

In [14]:
filtered_rain_gauges = filtered_rain_gauges[["codEstacao", "datahora", "valorMedida"]]

In [15]:
rain_gauge_812A = filtered_rain_gauges[filtered_rain_gauges["codEstacao"] == "355030812A"]

In [16]:
rain_gauge_833A = filtered_rain_gauges[filtered_rain_gauges["codEstacao"] == "355030833A"]

In [17]:
time_series = pd.merge(date_range, rain_gauge_812A, left_on="time", right_on="datahora", how="left")

In [18]:
time_series = time_series.drop(columns=["codEstacao", "datahora"])
time_series = time_series.rename(columns={"valorMedida": "812A"})
time_series["812A"] = time_series["812A"].fillna(0)

In [19]:
time_series = pd.merge(time_series, rain_gauge_833A, left_on="time", right_on="datahora", how="left")

In [20]:
time_series = time_series.drop(columns=["codEstacao", "datahora"])
time_series = time_series.rename(columns={"valorMedida": "833A"})
time_series["833A"] = time_series["833A"].fillna(0)

In [21]:
time_series.head()

,time,812A,833A
0,2019-01-01 00:00:00,0.0,0.0
1,2019-01-01 00:10:00,0.0,0.0
2,2019-01-01 00:20:00,0.0,0.0
3,2019-01-01 00:30:00,0.0,0.0
4,2019-01-01 00:40:00,0.0,0.0


## Tweet data

In [22]:
tweets_812A_jan = pd.read_csv(TWEETS_812A_JAN)
tweets_812A_fev = pd.read_csv(TWEETS_812A_FEV)
tweets_812_mar = pd.read_csv(TWEETS_812A_MAR)
tweets_812 = pd.concat([tweets_812A_jan, tweets_812A_fev, tweets_812_mar])

In [23]:
tweets_812 = tweets_812[["date", "text"]]
tweets_812["rain_gauge"] = "812A_tweets"

In [24]:
tweets_833 = pd.read_csv(TWEETS_833A)

In [25]:
tweets_833 = tweets_833[["date", "text"]]
tweets_833["rain_gauge"] = "833A_tweets"

In [26]:
tweets = pd.concat([tweets_812, tweets_833])

In [27]:
pattern = "|".join(KEYWORDS)

In [28]:
filtered_data = tweets[tweets["text"].str.contains(pattern, case=False, na=False)]

In [29]:
filtered_data["date"] = pd.to_datetime(filtered_data["date"])

C:\Users\Jefferson\AppData\Local\Temp\ipykernel_18652\721784688.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data["date"] = pd.to_datetime(filtered_data["date"])


In [30]:
filtered_data["date"] = filtered_data["date"].dt.round("10min")

C:\Users\Jefferson\AppData\Local\Temp\ipykernel_18652\2042031006.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data["date"] = filtered_data["date"].dt.round("10min")


In [31]:
tweet_counts = filtered_data.groupby(['date', 'rain_gauge']).size().reset_index(name='count')
result = tweet_counts.pivot(index='date', columns='rain_gauge', values='count')
result = result.fillna(0).reset_index()

In [32]:
total_data = pd.merge(time_series, result, left_on="time", right_on="date", how="left")

In [33]:
total_data["812A_tweets"] = total_data["812A_tweets"].fillna(0)
total_data["833A_tweets"] = total_data["833A_tweets"].fillna(0)

In [34]:
total_data = total_data.drop(columns="date")

## Floods

In [35]:
floods_812A = pd.read_csv(FLOODS_812A)
floods_812A["DATA"] = pd.to_datetime(floods_812A["DATA"])
floods_812A = floods_812A[floods_812A["DATA"] <= "2019-03-31"]
floods_812A = floods_812A[["DATA", "H_INICIO", "H_FIM"]]
floods_812A["rain_gauge"] = "812A"

In [36]:
floods_833A = pd.read_csv(FLOODS_833A)
floods_833A["DATA"] = pd.to_datetime(floods_833A["DATA"])
floods_833A = floods_833A[floods_833A["DATA"] <= "2019-03-31"]
floods_833A = floods_833A[["DATA", "H_INICIO", "H_FIM"]]
floods_833A["rain_gauge"] = "833A"

In [37]:
floods = pd.concat([floods_812A, floods_833A])

In [38]:
floods['DATA'] = floods['DATA'].astype(str)
floods['H_INICIO'] = floods['H_INICIO'].astype(str)
floods["date_start"] = pd.to_datetime(floods['DATA'] + ' ' + floods['H_INICIO'])

In [39]:
floods['DATA'] = floods['DATA'].astype(str)
floods['H_FIM'] = floods['H_FIM'].astype(str)
floods["date_end"] = pd.to_datetime(floods['DATA'] + ' ' + floods['H_FIM'])

In [40]:
floods = floods[["date_start", "date_end", "rain_gauge"]]

In [41]:
floods['date_start'] = floods['date_start'].dt.round('10min')
floods['date_end'] = floods['date_end'].dt.round('10min')

In [42]:
for gauge in floods['rain_gauge'].unique():
    total_data[f"{gauge}_flood"] = 0

In [43]:
for _, row in floods.iterrows():
    mask = (total_data['time'] >= row['date_start']) & (total_data['time'] <= row['date_end'])
    total_data.loc[mask, f"{row['rain_gauge']}_flood"] = 1

In [44]:
total_data.to_csv("data/results/time_data.csv", sep=";")

## TODO: Analyze data below

In [45]:
tweets_2016 = pd.read_csv("data/tweets/tweets.csv")

In [46]:
tweets_2016["created_at"].min()

'2016-11-07 10:40:16'

In [47]:
tweets_2016["created_at"].max()

'2017-11-30 23:59:57'

In [49]:
tweets_2016 = tweets_2016[tweets_2016["text"].str.contains(pattern, case=False, na=False)].info()

<class 'pandas.core.frame.DataFrame'>
Index: 8831 entries, 113 to 2781142
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id_str       8831 non-null   int64  
 1   user_id_str  8831 non-null   float64
 2   created_at   8831 non-null   object 
 3   retweeted    8831 non-null   bool   
 4   place_name   8829 non-null   object 
 5   text         8831 non-null   object 
 6   lon          8831 non-null   float64
 7   lat          8831 non-null   float64
dtypes: bool(1), float64(3), int64(1), object(3)
memory usage: 560.6+ KB


In [50]:
tweets_2016.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2781533 entries, 0 to 2781532
Data columns (total 8 columns):
 #   Column       Dtype  
---  ------       -----  
 0   id_str       int64  
 1   user_id_str  float64
 2   created_at   object 
 3   retweeted    bool   
 4   place_name   object 
 5   text         object 
 6   lon          float64
 7   lat          float64
dtypes: bool(1), float64(3), int64(1), object(3)
memory usage: 151.2+ MB
